In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping

# Metadata dosyasını yükle
metadata_path = './ham10000/HAM10000_metadata.csv'
skin_data = pd.read_csv(metadata_path)
skin_data.head()


In [ ]:
!pip install -q kaggle


In [ ]:
from google.colab import files
files.upload()  # kaggle.json dosyasını yükle


In [ ]:
!pip install -q scikit-learn
!pip install -q opendatasets


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
!unzip -q skin-cancer-mnist-ham10000.zip -d ham10000


In [ ]:
!ls ham10000


In [ ]:
# İki ayrı klasördeki görüntülerin dizinleri
image_dir1 = './ham10000/HAM10000_images_part_1'
image_dir2 = './ham10000/HAM10000_images_part_2'

# Görüntüleri yüklemek için boş liste oluştur
image_data = []

for image_id in skin_data['image_id']:
    image_path = os.path.join(image_dir1, image_id + '.jpg')
    if not os.path.exists(image_path):
        image_path = os.path.join(image_dir2, image_id + '.jpg')

    # Resmi oku, BGR'den RGB'ye çevir ve 28x28 boyutuna yeniden boyutlandır
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (28, 28))
    image_data.append(image)

image_data = np.array(image_data)
print(f"Yüklenen resim sayısı: {image_data.shape[0]}")


In [ ]:
plt.figure(figsize=(15, 3))

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(image_data[i])
    plt.axis('off')
    plt.title(f"Image {i + 1}")

plt.tight_layout()
plt.show()


In [ ]:
# Sınıf ID'leri ve açıklamaları
classes = {
    0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),
    1: ('bcc', 'Basal cell carcinoma'),
    2: ('bkl', 'Benign keratosis-like lesions'),
    3: ('df', 'Dermatofibroma'),
    4: ('nv', 'Melanocytic nevi'),
    5: ('vasc', 'Pyogenic granulomas and hemorrhage'),
    6: ('mel', 'Melanoma')
}

# Metadata'daki 'dx' sütunundaki sınıf etiketlerini sayısal hale çevir
label_map = {v[0]: k for k, v in classes.items()}
skin_data['label'] = skin_data['dx'].map(label_map)

print(skin_data[['image_id', 'dx', 'label']].head())


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=skin_data, palette='viridis')
plt.xticks(ticks=range(len(classes)), labels=[v[0] for v in classes.values()])
plt.xlabel('Sınıflar')
plt.ylabel('Örnek Sayısı')
plt.title('HAM10000 Veri Seti Sınıf Dağılımı')
plt.show()


In [ ]:
# Görüntüleri 0-1 aralığına ölçekle
image_data = image_data / 255.0

# Görüntüleri flatten yap (tek boyutlu vektör haline getir)
image_data_flat = image_data.reshape(image_data.shape[0], -1)

# İstersen skin_data'daki diğer özellikleri kullanabilirsin ama şimdilik sadece görüntülerle çalışıyoruz
# Özellik matrisi (X) ve etiketler (y)
X = image_data_flat
y = skin_data['label'].values

print(f'Özellik matrisi şekli: {X.shape}')
print(f'Etiket vektörü şekli: {y.shape}')


In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

print(f"Orijinal veri seti boyutu: {X.shape[0]}")
print(f"Dengesizleştirilmiş veri seti boyutu: {X_resampled.shape[0]}")

# Yeniden etiket dağılımını çiz
plt.figure(figsize=(10, 6))
sns.countplot(x=y_resampled, palette='viridis')
plt.xlabel('Sınıflar')
plt.ylabel('Örnek Sayısı')
plt.title('RandomOverSampler Sonrası Sınıf Dağılımı')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Örnek sınıf dağılımı grafiği
plt.figure(figsize=(10, 6))
sns.countplot(x=y_resampled, palette='viridis')
plt.xlabel('Sınıflar')
plt.ylabel('Örnek Sayısı')
plt.title('RandomOverSampler Sonrası Sınıf Dağılımı')
plt.show()

# Tabular veri histogram örneği
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(X_resampled[:1000].flatten(), bins=50, color='skyblue', edgecolor='black')
plt.title("Original Tabular Data Distribution")
plt.xlabel("Values")
plt.ylabel("Frequency")

x_normalized = (X_resampled[:1000] - np.mean(X_resampled[:1000], axis=0)) / np.std(X_resampled[:1000], axis=0)
plt.subplot(1, 2, 2)
plt.hist(x_normalized.flatten(), bins=50, color='salmon', edgecolor='black')
plt.title("Normalized Tabular Data Distribution")
plt.xlabel("Normalized Values")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

random_image_idx = np.random.randint(0, len(image_data))
image_original = image_data[random_image_idx]
image_flattened = image_data_flat[random_image_idx]

# Normalize image_original to 0-1 range for proper visualization
img_norm = (image_original - image_original.min()) / (image_original.max() - image_original.min() + 1e-7)

plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(img_norm)
plt.title("Image Resize 28x28 (Original Normalized)")
plt.axis('off')

plt.subplot(1, 2, 2)
img_reshaped = image_flattened.reshape(28, 28, 3)
img_min, img_max = img_reshaped.min(), img_reshaped.max()
img_norm2 = (img_reshaped - img_min) / (img_max - img_min + 1e-7)
plt.imshow(img_norm2)
plt.title("Flattened Image (Reshaped & Normalized)")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist

# Varsayılan olarak x, y tanımlı değilse mnist dataset kullanılır
# (sen kendi x,y kullanacaksan onları bu kısımda hazırla)

# (İstersen aşağıdaki yorum satırlarını açabilirsin)
# print(f"x shape: {x.shape}")
# print(f"y shape: {y.shape}")
# min_samples = min(len(x), len(y))
# x = x[:min_samples]
# y = y[:min_samples]

# Eğitim ve test verisini ayır
# X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Sınıf dağılımını görselleştir (eğitim öncesi)
# plt.figure(figsize=(8, 6))
# sns.countplot(x=Y_train)
# plt.title("Class Distribution (Before Balancing)")
# plt.xlabel("Class")
# plt.ylabel("Count")
# plt.show()

# Eğer kendi veri setini kullanıyorsan yukarıdaki satırları aktif et.

# MNIST verisini yüklüyoruz (eğer kendi veri setin yoksa)
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Veriyi normalize et ve reshape et (SMOTE için düzleştir)
X_train_flattened = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test_flattened = X_test.reshape(X_test.shape[0], -1) / 255.0

# SMOTE ile eğitim verisini dengele
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train)

# Sınıf dağılımını görselleştir (dengeleme sonrası)
plt.figure(figsize=(8, 6))
sns.countplot(x=Y_train_resampled)
plt.title("Class Distribution (After SMOTE)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

# Tekrar 28x28 şeklinde reshape et
X_train_resampled = X_train_resampled.reshape(-1, 28, 28)

# 7 sınıf filtresi uygula (0-6 arası sınıflar)
valid_indices_train = Y_train_resampled <= 6
X_train_resampled = X_train_resampled[valid_indices_train]
Y_train_resampled = Y_train_resampled[valid_indices_train]

valid_indices_test = Y_test <= 6
X_test_filtered = X_test_flattened[valid_indices_test].reshape(-1, 28, 28)
Y_test_filtered = Y_test[valid_indices_test]

# Modeli oluştur
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=3,
                               restore_best_weights=True,
                               verbose=1)

# Modeli eğit
history = model.fit(X_train_resampled, Y_train_resampled,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=20,
                    callbacks=[early_stopping])

# Eğitim & doğrulama grafiklerini çiz
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Test verisinde modeli değerlendir
test_loss, test_acc = model.evaluate(X_test_filtered, Y_test_filtered, verbose=2)
print(f"Test Kaybı: {test_loss:.4f}")
print(f"Test Doğruluğu: {test_acc:.4f}")

# Test seti üzerinde tahmin yap
Y_pred_probs = model.predict(X_test_filtered)
Y_pred = np.argmax(Y_pred_probs, axis=1)

# Sınıf bazlı raporu yazdır
print("Sınıf Bazlı Performans Raporu:")
print(classification_report(Y_test_filtered, Y_pred))

# Karışıklık matrisi hesapla ve görselleştir
cm = confusion_matrix(Y_test_filtered, Y_pred)

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Karışıklık Matrisi")
plt.colorbar()
tick_marks = np.arange(7)
plt.xticks(tick_marks, tick_marks)
plt.yticks(tick_marks, tick_marks)
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')

thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             ha="center", va="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Model tahminleri
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Rastgele 10 örnek seç
indices = np.random.choice(range(len(X_test)), size=10, replace=False)

# Görüntüler (image_data'nın shape ve dtype'ı önemli, tip olarak uint8 olmalı)
sample_images = image_data[indices]

# True label alma - Pandas Series ise .iloc ile, değilse direkt index ile al
true_labels = Y_test.iloc[indices].values if isinstance(Y_test, pd.Series) else Y_test[indices]
predicted_labels_sample = predicted_labels[indices]

# Görüntüleri 0-1 aralığına normalize eden fonksiyon
def normalize_image(img):
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max - img_min == 0:
        return img
    return (img - img_min) / (img_max - img_min)

# Görselleri normalize et (özellikle float ve küçük değerler varsa)
for i in range(len(sample_images)):
    sample_images[i] = normalize_image(sample_images[i])

plt.figure(figsize=(15, 8))
for i, idx in enumerate(indices):
    plt.subplot(2, 5, i + 1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {true_labels[i]}\nPred: {predicted_labels_sample[i]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
model.save('my_model.keras')



In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_model.keras')


In [ ]:
import os

print(os.getcwd())
